# London Open Trend Follow — Backtesting Analysis

This notebook backtests a simple intraday trend-following strategy based on the direction of the first London session candle.

## Strategy Overview

At the close of the **07:00 UTC** candle (London open), determine the session direction and enter on the next bar. Exit intraday — no overnight positions, no overnight fees.

### Entry Rule
- **LONG**: 07:00 UTC candle closes bullish (close > open) → enter at 08:00 UTC open
- **SHORT**: 07:00 UTC candle closes bearish (close < open) → enter at 08:00 UTC open

### Exit Rule
- Close position at **17:00 UTC open** (configurable)
- One trade per day maximum
- No stop loss / take profit — pure time-based exit (starting point; SL/TP added in optimisation)

### Research Background
Grid search over 258 trading days of EUR/USD H1 2025 data showed:

| Entry | Exit | Win Rate | Avg P&L | Total P&L |
|-------|------|----------|---------|-----------|
| 08:00 UTC (after 07:00 signal) | 17:00 UTC | ~52–63% | +5–16 pips | +1,200–4,000 pips |

The 07:00 signal + 17:00 exit was the single best combination found.

### Files
- Data: `data/historical/EUR_USD/EUR_USD_H1_20250101_20251231.csv`

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest, Strategy
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print('✓ Imports successful')

## 2. Configuration

In [ ]:
INSTRUMENT  = 'EUR_USD'
GRANULARITY = 'H1'
FROM_DATE   = '20250101'
TO_DATE     = '20251231'

INITIAL_CASH = 10_000
COMMISSION   = 0.0001   # ~1 pip round-trip

DATA_PATH = Path('../data/historical') / INSTRUMENT / f'{INSTRUMENT}_{GRANULARITY}_{FROM_DATE}_{TO_DATE}.csv'

print(f'Instrument : {INSTRUMENT}')
print(f'Timeframe  : {GRANULARITY}')
print(f'Period     : {FROM_DATE} – {TO_DATE}')
print(f'Capital    : ${INITIAL_CASH:,.0f}')
print(f'Data path  : {DATA_PATH}')
print(f'Data exists: {DATA_PATH.exists()}')

## 3. Load Data

In [ ]:
def load_h1_data(path: Path) -> pd.DataFrame:
    """Load OANDA CSV, return DataFrame formatted for backtesting.py."""
    df = pd.read_csv(path, comment='#', parse_dates=['time'], index_col='time')
    df.index = pd.to_datetime(df.index, utc=True)
    df.columns = [c.capitalize() for c in df.columns]
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]


df = load_h1_data(DATA_PATH)

print(f'Loaded {len(df):,} H1 candles')
print(f'Range  : {df.index.min()} → {df.index.max()}')
print(f'Trading days: {df.index.normalize().nunique()}')
df.head()

## 4. Exploratory Analysis

Understand hourly volatility and directional bias before writing any strategy code.

In [ ]:
raw = pd.read_csv(DATA_PATH, comment='#', parse_dates=['time'])
raw['time'] = pd.to_datetime(raw['time'], utc=True)
raw['hour']   = raw['time'].dt.hour
raw['range']  = (raw['high'] - raw['low']) * 10_000   # pips
raw['ret']    = (raw['close'] - raw['open']) * 10_000  # pips
raw['bull']   = raw['ret'] > 0

hourly = raw.groupby('hour').agg(
    avg_range  = ('range', 'mean'),
    avg_ret    = ('ret',   'mean'),
    win_rate   = ('bull',  'mean'),
).round(2)
hourly['win_rate'] = (hourly['win_rate'] * 100).round(1)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(16, 4))
trading_hours = hourly.loc[6:21]

axes[0].bar(trading_hours.index, trading_hours['avg_range'], color='steelblue')
axes[0].axvline(7, color='red', linestyle='--', label='07:00 UTC')
axes[0].axvline(13, color='orange', linestyle='--', label='13:00 UTC (NY)')
axes[0].set_title('Avg Hourly Range (pips)')
axes[0].set_xlabel('Hour (UTC)')
axes[0].legend(fontsize=8)

colors = ['green' if v > 0 else 'red' for v in trading_hours['avg_ret']]
axes[1].bar(trading_hours.index, trading_hours['avg_ret'], color=colors)
axes[1].axhline(0, color='black', linewidth=0.8)
axes[1].axvline(7, color='red', linestyle='--')
axes[1].set_title('Avg Directional Return (pips)')
axes[1].set_xlabel('Hour (UTC)')

axes[2].bar(trading_hours.index, trading_hours['win_rate'], color='purple', alpha=0.7)
axes[2].axhline(50, color='black', linewidth=0.8, linestyle='--', label='50%')
axes[2].axvline(7, color='red', linestyle='--')
axes[2].set_title('Bullish Candle Win Rate (%)')
axes[2].set_xlabel('Hour (UTC)')
axes[2].legend(fontsize=8)

plt.suptitle('EUR/USD H1 2025 — Hourly Statistics', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

print('\nHourly stats (07:00–20:00 UTC):')
display(hourly.loc[7:20])

### 07:00 Candle — Signal Quality Check

How well does the 07:00 candle predict the rest of the day?

In [ ]:
raw['date'] = raw['time'].dt.date

# For each day: get 07:00 signal and net move from 08:00 open to 17:00 close
signal_rows = []
for date, day in raw.groupby('date'):
    day = day.sort_values('hour')
    sig  = day[day['hour'] == 7]
    entry = day[day['hour'] == 8]
    exit_ = day[day['hour'] == 17]
    if len(sig) == 0 or len(entry) == 0 or len(exit_) == 0:
        continue
    signal_dir  = 1 if sig.iloc[0]['ret'] > 0 else -1
    entry_price = entry.iloc[0]['open']
    exit_price  = exit_.iloc[0]['open']
    trade_pnl   = signal_dir * (exit_price - entry_price) * 10_000
    signal_rows.append({
        'date': date,
        'dow': pd.Timestamp(date).day_name(),
        'signal_dir': signal_dir,
        'signal_pips': sig.iloc[0]['ret'],
        'trade_pnl': trade_pnl,
        'win': trade_pnl > 0,
    })

signals = pd.DataFrame(signal_rows)

print(f'Trading days with full data: {len(signals)}')
print(f"Bullish signals: {(signals['signal_dir']==1).sum()}")
print(f"Bearish signals: {(signals['signal_dir']==-1).sum()}")
print(f"\nOverall win rate : {signals['win'].mean()*100:.1f}%")
print(f"Avg P&L per trade: {signals['trade_pnl'].mean():.1f} pips")
print(f"Total P&L        : {signals['trade_pnl'].sum():.1f} pips")

# Distribution of trade P&L
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].hist(signals['trade_pnl'], bins=30, edgecolor='black', color='steelblue', alpha=0.7)
axes[0].axvline(0, color='red', linestyle='--')
axes[0].axvline(signals['trade_pnl'].mean(), color='green', linestyle='--', label=f"Mean = {signals['trade_pnl'].mean():.1f}")
axes[0].set_title('Trade P&L Distribution (pips)  [08:00→17:00 UTC]')
axes[0].set_xlabel('P&L (pips)')
axes[0].legend()

cumulative = signals['trade_pnl'].cumsum()
axes[1].plot(range(len(cumulative)), cumulative, color='darkblue', linewidth=2)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_title('Cumulative P&L (pips)  [raw analysis, no position sizing]')
axes[1].set_xlabel('Trade #')
axes[1].set_ylabel('Cumulative pips')

plt.tight_layout()
plt.show()

### Day-of-Week Breakdown

In [ ]:
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
dow_stats = signals.groupby('dow').agg(
    trades    = ('trade_pnl', 'count'),
    win_rate  = ('win',       lambda x: f"{x.mean()*100:.1f}%"),
    avg_pnl   = ('trade_pnl', lambda x: round(x.mean(), 1)),
    total_pnl = ('trade_pnl', lambda x: round(x.sum(), 1)),
).reindex(dow_order)

print('Day-of-week breakdown (08:00→17:00 UTC):')
display(dow_stats)

# Bar chart
dow_avg = signals.groupby('dow')['trade_pnl'].mean().reindex(dow_order)
colors = ['green' if v > 0 else 'red' for v in dow_avg]
plt.figure(figsize=(8, 4))
plt.bar(dow_avg.index, dow_avg.values, color=colors, edgecolor='black', alpha=0.8)
plt.axhline(0, color='black', linewidth=0.8)
plt.title('Avg P&L by Day of Week (pips)')
plt.ylabel('Avg P&L (pips)')
plt.tight_layout()
plt.show()

## 5. Strategy Implementation

Three variants:
- **Base**: enter at 08:00 UTC (after 07:00 candle closes), exit at `exit_hour` open
- **DayFilter**: same, but only trade Monday and Wednesday (best days from analysis)
- **WithSL**: adds a stop loss and take profit in pips

In [ ]:
class LondonOpenTrendFollow(Strategy):
    """
    Enter at 08:00 UTC open in the direction of the 07:00 UTC candle.
    Exit at exit_hour UTC open (time-based, no SL/TP).
    One trade per day.
    """
    exit_hour = 17   # UTC hour to close position (executes at that hour's open)

    def init(self):
        pass

    def next(self):
        bar_hour = self.data.index[-1].hour

        # --- Exit: close at exit_hour open ---
        if bar_hour == self.exit_hour and self.position:
            self.position.close()
            return

        if self.position:
            return

        # --- Entry signal: end of 07:00 UTC candle ---
        # self.buy() / self.sell() execute at next bar's open (08:00 UTC)
        if bar_hour == 7:
            o = self.data.Open[-1]
            c = self.data.Close[-1]
            if c > o:
                self.buy()
            elif c < o:
                self.sell()


class LondonOpenDayFilter(Strategy):
    """
    Same as LondonOpenTrendFollow but only trades Monday (0) and Wednesday (2).
    """
    exit_hour = 17
    trade_days = (0, 2)   # Mon=0, Wed=2

    def init(self):
        pass

    def next(self):
        bar_time = self.data.index[-1]
        bar_hour = bar_time.hour

        if bar_hour == self.exit_hour and self.position:
            self.position.close()
            return

        if self.position:
            return

        if bar_hour == 7 and bar_time.weekday() in self.trade_days:
            o = self.data.Open[-1]
            c = self.data.Close[-1]
            if c > o:
                self.buy()
            elif c < o:
                self.sell()


class LondonOpenWithSL(Strategy):
    """
    London Open + stop loss and take profit in pips.
    Still exits at exit_hour if neither SL nor TP has been hit.
    """
    exit_hour  = 17
    sl_pips    = 20   # stop loss distance in pips
    tp_pips    = 40   # take profit distance in pips

    def init(self):
        pass

    def next(self):
        bar_hour = self.data.index[-1].hour

        if bar_hour == self.exit_hour and self.position:
            self.position.close()
            return

        if self.position:
            return

        if bar_hour == 7:
            o = self.data.Open[-1]
            c = self.data.Close[-1]
            pip = 0.0001
            sl  = self.sl_pips * pip
            tp  = self.tp_pips * pip
            if c > o:
                self.buy(sl=c - sl, tp=c + tp)
            elif c < o:
                self.sell(sl=c + sl, tp=c - tp)


print('✓ Strategy classes defined')
print('  - LondonOpenTrendFollow : base strategy')
print('  - LondonOpenDayFilter   : Mon & Wed only')
print('  - LondonOpenWithSL      : with stop loss / take profit')

## 6. Backtest — Base Strategy

In [ ]:
bt_base = Backtest(df, LondonOpenTrendFollow, cash=INITIAL_CASH, commission=COMMISSION)
stats_base = bt_base.run()

print('=' * 70)
print('BASE STRATEGY  —  07:00 signal, 08:00 entry, 17:00 exit')
print('=' * 70)
print(stats_base)

## 7. Backtest — Day Filter (Mon & Wed only)

In [ ]:
bt_dow = Backtest(df, LondonOpenDayFilter, cash=INITIAL_CASH, commission=COMMISSION)
stats_dow = bt_dow.run()

print('=' * 70)
print('DAY FILTER STRATEGY  —  Mon & Wed only')
print('=' * 70)
print(stats_dow)

## 8. Backtest — With Stop Loss & Take Profit

In [ ]:
bt_sl = Backtest(df, LondonOpenWithSL, cash=INITIAL_CASH, commission=COMMISSION)
stats_sl = bt_sl.run()

print('=' * 70)
print('SL/TP STRATEGY  —  20 pip SL, 40 pip TP (1:2 R:R), 17:00 time exit')
print('=' * 70)
print(stats_sl)

## 9. Strategy Comparison

In [ ]:
def fmt_stats(s, label):
    return {
        'Strategy':       label,
        'Return (%)':     f"{s['Return [%]']:.2f}%",
        'Sharpe':         f"{s['Sharpe Ratio']:.2f}",
        'Max DD (%)':     f"{s['Max. Drawdown [%]']:.2f}%",
        'Win Rate (%)':   f"{s['Win Rate [%]']:.1f}%",
        '# Trades':       int(s['# Trades']),
        'Final Equity':   f"${s['Equity Final [$]']:,.2f}",
    }

comparison = pd.DataFrame([
    fmt_stats(stats_base, 'Base (no SL/TP)'),
    fmt_stats(stats_dow,  'Mon & Wed filter'),
    fmt_stats(stats_sl,   'SL 20 / TP 40 pips'),
]).set_index('Strategy')

print('Strategy Comparison:')
display(comparison)

# Overlay equity curves
fig = go.Figure()
for stats, label, color in [
    (stats_base, 'Base', 'royalblue'),
    (stats_dow,  'Mon & Wed only', 'green'),
    (stats_sl,   'SL20/TP40', 'darkorange'),
]:
    eq = stats['_equity_curve']
    fig.add_trace(go.Scatter(x=eq.index, y=eq['Equity'], name=label,
                             line=dict(color=color, width=2)))

fig.add_hline(y=INITIAL_CASH, line_dash='dash', line_color='gray', annotation_text='Initial capital')
fig.update_layout(
    title='London Open Trend Follow — Equity Curve Comparison',
    xaxis_title='Date', yaxis_title='Equity ($)',
    height=500, hovermode='x unified'
)
fig.show()

## 10. Trade Analysis — Base Strategy

In [ ]:
trades = stats_base['_trades'].copy()

if len(trades) == 0:
    print('No trades executed.')
else:
    trades['Direction'] = trades['Size'].apply(lambda x: 'Long' if x > 0 else 'Short')
    trades['Duration_h'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds() / 3600
    trades['PnL_pips'] = trades['PnL'] / (INITIAL_CASH / 10_000) * 10_000  # approximate

    wins  = trades[trades['PnL'] > 0]
    losses = trades[trades['PnL'] < 0]

    print(f'Total trades : {len(trades)}')
    print(f'Winning      : {len(wins)} ({len(wins)/len(trades)*100:.1f}%)')
    print(f'Losing       : {len(losses)} ({len(losses)/len(trades)*100:.1f}%)')
    print(f'Long / Short : {len(trades[trades.Direction=="Long"])} / {len(trades[trades.Direction=="Short"])}')
    print(f'Avg duration : {trades["Duration_h"].mean():.1f} hours')
    print()
    if len(wins):
        print(f'Avg win  : ${wins["PnL"].mean():.2f}   Best : ${wins["PnL"].max():.2f}')
    if len(losses):
        print(f'Avg loss : ${losses["PnL"].mean():.2f}  Worst: ${losses["PnL"].min():.2f}')

    # Charts
    fig, axes = plt.subplots(1, 3, figsize=(16, 4))

    axes[0].hist(trades['PnL'], bins=25, edgecolor='black', color='steelblue', alpha=0.8)
    axes[0].axvline(0, color='red', linestyle='--')
    axes[0].set_title('P&L Distribution ($)')
    axes[0].set_xlabel('P&L ($)')

    cumulative = trades['PnL'].cumsum()
    axes[1].plot(range(len(cumulative)), cumulative, linewidth=2, color='darkblue')
    axes[1].axhline(0, color='red', linestyle='--')
    axes[1].set_title('Cumulative P&L ($)')
    axes[1].set_xlabel('Trade #')

    direction_counts = trades['Direction'].value_counts()
    axes[2].bar(direction_counts.index, direction_counts.values,
                color=['green', 'red'], edgecolor='black', alpha=0.8)
    axes[2].set_title('Long vs Short Trade Count')

    plt.tight_layout()
    plt.show()

    print('\nFirst 10 trades:')
    display(trades[['EntryTime', 'ExitTime', 'Direction', 'EntryPrice', 'ExitPrice', 'PnL', 'Duration_h']].head(10))

## 11. Parameter Optimisation — Exit Hour

Test exit hours from 14:00 to 21:00 UTC to find the best intraday close time.

In [ ]:
exit_results = []

for exit_h in range(14, 22):
    bt = Backtest(df, LondonOpenTrendFollow, cash=INITIAL_CASH, commission=COMMISSION)
    s = bt.run(exit_hour=exit_h)
    exit_results.append({
        'exit_hour': exit_h,
        'n_trades':   int(s['# Trades']),
        'win_rate':   round(s['Win Rate [%]'], 1),
        'return_pct': round(s['Return [%]'], 2),
        'sharpe':     round(s['Sharpe Ratio'], 2),
        'max_dd':     round(s['Max. Drawdown [%]'], 2),
        'final_eq':   round(s['Equity Final [$]'], 2),
    })

exit_df = pd.DataFrame(exit_results).set_index('exit_hour')
print('Exit hour optimisation:')
display(exit_df)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].plot(exit_df.index, exit_df['win_rate'], marker='o', color='steelblue')
axes[0].set_title('Win Rate (%) by Exit Hour')
axes[0].set_xlabel('Exit Hour (UTC)')
axes[0].axhline(50, color='red', linestyle='--')

axes[1].plot(exit_df.index, exit_df['return_pct'], marker='o', color='green')
axes[1].set_title('Total Return (%) by Exit Hour')
axes[1].set_xlabel('Exit Hour (UTC)')

axes[2].plot(exit_df.index, exit_df['sharpe'], marker='o', color='darkorange')
axes[2].set_title('Sharpe Ratio by Exit Hour')
axes[2].set_xlabel('Exit Hour (UTC)')

plt.suptitle('Impact of Exit Hour on Strategy Performance', fontsize=12)
plt.tight_layout()
plt.show()

## 12. SL/TP Optimisation

In [ ]:
print('Optimising SL and TP pips for LondonOpenWithSL...')
print('(this may take ~1 minute)')

bt_sltp = Backtest(df, LondonOpenWithSL, cash=INITIAL_CASH, commission=COMMISSION)
stats_sltp_opt = bt_sltp.optimize(
    sl_pips=range(10, 41, 10),
    tp_pips=range(20, 81, 10),
    exit_hour=range(15, 20),
    maximize='Sharpe Ratio',
    constraint=lambda p: p.tp_pips > p.sl_pips,
    return_heatmap=False,
)

print('\n' + '=' * 70)
print('OPTIMISED SL/TP RESULTS')
print('=' * 70)
print(stats_sltp_opt)
print(f"\nBest SL  : {stats_sltp_opt._strategy.sl_pips} pips")
print(f"Best TP  : {stats_sltp_opt._strategy.tp_pips} pips")
print(f"Best exit: {stats_sltp_opt._strategy.exit_hour}:00 UTC")

## 13. Interactive Backtest Chart

In [ ]:
# Visualise the base strategy with backtesting.py built-in chart
# Opens in browser; set open_browser=False to suppress
bt_base.plot(open_browser=True)

## 14. Conclusions

### Key Findings

| Question | Answer |
|----------|--------|
| Does the 07:00 UTC candle predict intraday direction? | *(fill after running)* |
| Best exit hour? | *(fill after running)* |
| Does a Mon/Wed filter improve results? | *(fill after running)* |
| Does adding SL/TP help or hurt? | *(fill after running)* |

### Strategy Strengths
- One trade per day — simple to manage
- Fully exits intraday — zero overnight fee risk
- Signal is objective (candle direction, no indicator lag)
- London open is the highest-liquidity window for EUR/USD

### Strategy Weaknesses
- No trend filter — trades every day regardless of macro context
- No volatility filter — treats a 2-pip candle the same as a 20-pip candle
- In-sample only — **must validate on out-of-sample data** before live trading

### Recommended Next Steps
1. **Out-of-sample test**: run this notebook on 2024 data to check for overfitting
2. **Volatility filter**: only trade when the 07:00 candle range > X pips (skip flat opens)
3. **News filter**: skip days with high-impact ECB/Fed news at or before 10:00 UTC
4. **Paper trade**: run on OANDA demo for 30 days before committing real capital
5. **Position sizing**: risk 1% of equity per trade (not fixed lot size)

---
**Reminder**: In-sample backtest results are optimistic. Past performance does not guarantee future results.